In [88]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import pickle
#
import re
from nltk.corpus import stopwords # to remove words which add no value to the data
from nltk.stem.porter import PorterStemmer
#
from sklearn.feature_extraction.text import TfidfVectorizer # text to feature vectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.metrics import accuracy_score,classification_report
#
import tensorflow as tf
from tensorflow.keras.models import Sequential # model
from tensorflow.keras.layers import Dense # type of layer
from tensorflow.keras.activations import linear, relu, sigmoid,softmax #activation function
from tensorflow.keras import regularizers

In [89]:
import nltk
nltk.download('stopwords')

KeyboardInterrupt: 

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

data processing

x and y are present as x.np and y.np, can be loaded directly

In [ ]:
data=pd.read_csv(r'C:\Users\deban\Desktop\solo\fake_news_predictor\train.csv')

In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
data=data.fillna('')

In [ ]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
data['content']=data['author']+' '+data['text']

In [ ]:
data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,Daniel J. Flynn Ever get the feeling your life...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss Videos 15 Civilians Killed In ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Print \nAn Iranian woman has be...


In [ ]:
data.columns

Index(['id', 'title', 'author', 'text', 'label', 'content'], dtype='object')

## Stemming

convert to the root word

In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(words) for words in stemmed_content if not words in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

In [ ]:
data['content']=data['content'].apply(stemming)

In [ ]:
print(data['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn ever get feel life circl rounda...
2        consortiumnew com truth might get fire octob t...
3        jessica purkiss video civilian kill singl us a...
4        howard portnoy print iranian woman sentenc six...
                               ...                        
20795    jerom hudson rapper unload black celebr met do...
20796    benjamin hoffman green bay packer lost washing...
20797    michael j de la merc rachel abram maci today g...
20798    alex ansari nato russia hold parallel exercis ...
20799    david swanson david swanson author activist jo...
Name: content, Length: 20800, dtype: object


In [ ]:
x=data['content'].values
y=data['label'].values

In [ ]:
vecto=TfidfVectorizer()
x=vecto.fit_transform(x)

In [ ]:
np.save('x.npy', x)
np.save('y.npy', y)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,stratify=y,random_state=2)

In [ ]:
type(x_train)

scipy.sparse._csr.csr_matrix

In [ ]:
model= LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [ ]:
train_pred= model.predict(x_train)
test_pred= model.predict(x_test)
train_acc=accuracy_score(train_pred,y_train)
test_acc=accuracy_score(test_pred,y_test)
print("accuracy_score:\n",train_acc,test_acc)


accuracy_score:
 0.9764423076923077 0.9454326923076923


In [ ]:
def get_pr(pred):
    if(pred==0):
        print("News is true")
    else:
        print("News is fake")

In [ ]:
i=3
x_new=x_test[i]

pred=model.predict(x_new)
get_pr(pred)
get_pr(y_test[i])

News is fake
News is true


In [ ]:
# Assuming your trained model is stored in the variable 'model'
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)


In [ ]:
i=5
new_data=x_test[i]

pred== loaded_model.predict(new_data)
get_pr(pred)
get_pr(y_test[i])


News is fake
News is fake
